In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, FunctionTransformer, OneHotEncoder, StandardScaler

import sys 
import sys
sys.path.append('../pipeline_scripts')
from preprocessing_pipeline import *
import pandas as pd

In [2]:
# read in data 

train = pd.read_csv('../project_data/train_data.csv')
train.head()


test = pd.read_csv('../project_data/test_data.csv')
test.head()

/var/folders/nj/p20z0h9x3ssf6ll0_cdhh2yc0000gn/T/ipykernel_1625/3352684517.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../project_data/train_data.csv')


,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Number of Dependents
0,2022-12-24,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaN,INDEMNITY INSURANCE CO OF,...,IV,NaN,31.0,"FALL, SLIP OR TRIP, NOC",10.0,CONTUSION,54.0,LOWER LEG,10466,1
1,2022-11-20,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaN,A I U INSURANCE COMPANY,...,IV,NaN,75.0,FALLING OR FLYING OBJECT,10.0,CONTUSION,10.0,MULTIPLE HEAD INJURY,11691,1
2,2022-12-26,59,N,2023-01-02,N,0.0,1963.0,2022-12-31,NaN,AMGUARD INSURANCE COMPANY,...,III,NaN,68.0,STATIONARY OBJECT,49.0,SPRAIN OR TEAR,62.0,BUTTOCKS,10604,0
3,2022-12-28,55,N,2023-01-02,N,0.0,0.0,2023-01-02,NaN,INDEMNITY INS. OF N AMERICA,...,IV,NaN,25.0,FROM DIFFERENT LEVEL (ELEVATION),10.0,CONTUSION,53.0,KNEE,11411,6
4,2022-12-20,25,N,2023-01-02,N,0.0,1997.0,2022-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,IV,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,40.0,LACERATION,37.0,THUMB,11212,5


In [3]:
train.columns

Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage',
       'Birth Year', 'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'OIICS Nature of Injury Description',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents'],
      dtype='object')

In [4]:
# drop na values for Alternative Dispute Resolution 
train.dropna(subset= ['Alternative Dispute Resolution'], inplace = True)
train.dropna(subset= ['Accident Date'], inplace = True)
train.dropna(subset= ['Assembly Date'], inplace = True)

# set accident Dates and assembly Dates to datetime
train['Accident Date'] = pd.to_datetime(train['Accident Date'])
train['Assembly Date'] = pd.to_datetime(train['Assembly Date'])


# drop na values for Alternative Dispute Resolution 
test.dropna(subset= ['Alternative Dispute Resolution'], inplace = True)
test.dropna(subset= ['Accident Date'], inplace = True)
test.dropna(subset= ['Assembly Date'], inplace = True)

# set accident Dates and assembly Dates to datetime
test['Accident Date'] = pd.to_datetime(test['Accident Date'])
test['Assembly Date'] = pd.to_datetime(test['Assembly Date'])

In [5]:
# drop some values so we can test the pipeline
sorted(train['Alternative Dispute Resolution'].unique())

train['Alternative Dispute Resolution'].value_counts()

# drop the value U 
train = train[train['Alternative Dispute Resolution'] != 'U']


# drop some values so we can test the pipeline
sorted(test['Alternative Dispute Resolution'].unique())

test['Alternative Dispute Resolution'].value_counts()

# drop the value U 
test = test[test['Alternative Dispute Resolution'] != 'U']

In [6]:
train

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,...,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,...,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
4,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,...,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0
5,2019-12-26,67.0,N,2020-01-01,N,0.00,1952.0,2019-12-31,NaN,INDEMNITY INS. OF N AMERICA,...,31.0,"FALL, SLIP OR TRIP, NOC",10.0,CONTUSION,38.0,SHOULDER(S),11772,0.0,Not Work Related,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593451,2022-12-14,35.0,N,2022-12-30,N,0.00,1987.0,2022-12-30,NaN,STATE INSURANCE FUND,...,45.0,COLLISION OR SIDESWIPE WITH ANOTHER VEHICLE,10.0,CONTUSION,42.0,LOWER BACK AREA,10467,0.0,Not Work Related,4.0
593455,2022-12-15,33.0,N,2022-12-31,N,0.00,1989.0,2022-12-31,NaN,WESCO INSURANCE CO,...,74.0,"FELLOW WORKER, PATIENT OR OTHER PERSON",37.0,INFLAMMATION,35.0,HAND,11590,0.0,Not Work Related,6.0
593456,2022-12-13,61.0,N,2022-12-31,N,991.08,1961.0,2022-12-31,NaN,SECURITY NATIONAL INSURANCE CO,...,98.0,"CUMULATIVE, NOC",80.0,"ALL OTHER CUMULATIVE INJURY, NOC",34.0,WRIST,14227,0.0,Not Work Related,3.0
593457,2022-12-14,24.0,N,2022-12-31,N,0.00,1998.0,2022-12-31,NaN,TECHNOLOGY INSURANCE CO. INC.,...,59.0,USING TOOL OR MACHINERY,59.0,"ALL OTHER SPECIFIC INJURIES, NOC",55.0,ANKLE,10029,0.0,Not Work Related,5.0


In [7]:
class BinaryEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, binary_columns):
        self.binary_columns = binary_columns
        self.encodings = {}  # Store the mapping for each column

    # Fit method creates a dictionary which defines the mapping. 
    def fit(self, X, y):
        return self

    def transform(self, X):
        X = X.copy()  # Avoid modifying the original DataFrame
        for col in self.binary_columns:
            unique_values = sorted(X[col].unique())

            if len(unique_values) != 2:
                raise ValueError(f"Column '{col}' does not have exactly two unique values.")

            X[col] = X[col].map({unique_values[0]: 0, unique_values[1]: 1})

        return X

class MultipleTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self,
                # target_columns
                feature_column):
        # self.target_columns = target_columns
        self.feature_column = feature_column
        self.target_encoders = {}  # Store the target encoders for each column 

    def fit(self, X, y):
        X = X.copy()  # Avoid modifying the original DataFrame
        for unique_value in y.unique():
            
            # for every unique value there needs to be one new column
            y_binary = (y == unique_value).astype(int)

            target_encoder = TargetEncoder(cols=[self.feature_column])
            X[self.feature_column] = X[self.feature_column].astype('category')
            
            # fit the encoder
            target_encoder.fit(X[[self.feature_column]], y_binary)

            # safe fit in a dictionary
            self.target_encoders[unique_value] = target_encoder

        return self
    
    def transform(self, X):
        X = X.copy()  # Avoid modifying the original DataFrame
        for unique_value, encoder  in self.target_encoders.items():

            # then transform the validation set using the y_train binary target to avoid data leakage
            encoded_column = encoder.transform(X[[self.feature_column]])

            # Rename and add the encoded column to the DataFrame
            X[f'{self.feature_column}_encoded_{unique_value}'] = encoded_column

        return X
    # X contains the dataframe with the respective column that we want to encode 

class Days_between(BaseEstimator, TransformerMixin):
    def __init__(self, start_col, end_col):
        self.start_col = start_col
        self.end_col = end_col

    def fit(self, X, y):
        return self
    
    def transform(self, X):
        X = X.copy()  # Avoid modifying the original DataFrame
        X[f'Days_between_{self.end_col}_{self.start_col}'] = (X[self.end_col] - X[self.start_col]).dt.days
        return X
    

class DummyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, dummy_column, drop = 'first'):
        self.dummy_column = dummy_column
        self.drop = drop
        self.encoder = OneHotEncoder(drop = self.drop)

    def fit(self, X, y=None):
        X = X.copy()  # Avoid modifying the original DataFrame
        self.encoder.fit(X[[self.dummy_column]])
        return self

    def transform(self, X):
        X = X.copy()  # Avoid modifying the original DataFrame
        transformed_data = self.encoder.transform(X[[self.dummy_column]])
        transformed_df = pd.DataFrame(transformed_data.toarray(), columns=self.encoder.get_feature_names_out([self.dummy_column]))
        X = X.drop(columns=[self.dummy_column])
        X = pd.concat([X.reset_index(drop=True), transformed_df.reset_index(drop=True)], axis=1)
        return X

class ColumnMapper(BaseEstimator, TransformerMixin):
    def __init__(self, column_name, mapping_dict = None, drop_original = True):
        self.mapping_dict = mapping_dict
        self.column_name = column_name
        self.drop_original = drop_original

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()  # Avoid modifying the original DataFrame

        if self.drop_original:
            X[self.column_name] = X[self.column_name].map(self.mapping_dict)
    
        else:
            X[f'{self.column_name}_mapped'] = X[self.column_name].map(self.mapping_dict)

        return X
    

class NAIndicatorEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()  # Avoid modifying the original DataFrame

        # Encode the column as 1 for not NA and 0 for NA
        X[self.column_name] = X[self.column_name].notna().astype(int)
        
        return X

In [8]:
# Define the mapping
carrier_type_mapping = {
    '1A. PRIVATE': 'Private Insurance Carrier',
    '2A. SIF': 'State Insurance Fund',
    '3A. SELF PUBLIC': 'Self-insured Public Entity',
    '4A. SELF PRIVATE': 'Self-insured Private Entity',
    '5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)': 'Special Funds',
    '5C. SPECIAL FUND - POI CARRIER WCB MENANDS': 'Special Funds',
    '5D. SPECIAL FUND - UNKNOWN': 'Special Funds',
    'UNKNOWN': 'Unknown'
}

In [9]:
y_train = train['Claim Injury Type']
y_train.unique()

array(['2. NON-COMP', '4. TEMPORARY', '3. MED ONLY', '5. PPD SCH LOSS',
       '6. PPD NSL', '1. CANCELLED', '8. DEATH', '7. PTD'], dtype=object)

In [10]:
train['Attorney/Representative'] = train['Attorney/Representative'].astype(str)


In [11]:
print(type(train['Attorney/Representative']))

<class 'pandas.core.series.Series'>


In [12]:
print(type(train['Attorney/Representative']))

<class 'pandas.core.series.Series'>


In [ ]:
# versteh nicht wieso die classes manchmal inplace umsetzen und manchmal nicht siehe DummyEncoder class???
binary_columns_list = ['Alternative Dispute Resolution', 'COVID-19 Indicator']
target_encoder_list = ['Industry Code',
                    'WCIO Cause of Injury Code',
                    'WCIO Nature of Injury Code',
                    'WCIO Part Of Body Code']

# Define the mapping
carrier_type_mapping = {
    '1A. PRIVATE': 'Private Insurance Carrier',
    '2A. SIF': 'State Insurance Fund',
    '3A. SELF PUBLIC': 'Self-insured Public Entity',
    '4A. SELF PRIVATE': 'Self-insured Private Entity',
    '5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)': 'Special Funds',
    '5C. SPECIAL FUND - POI CARRIER WCB MENANDS': 'Special Funds',
    '5D. SPECIAL FUND - UNKNOWN': 'Special Funds',
    'UNKNOWN': 'Unknown'
}




preprocessing_pipeline = Pipeline(steps=[
     # Apply the binary  transformation for the list 
    ('binary_encoder_bin', BinaryEncoder(binary_columns=binary_columns_list)),

    # Apply the target encoder for following columns
    ('target_encoder_in', MultipleTargetEncoder(feature_column = 'Industry Code')),
    ('target_encoder_coi', MultipleTargetEncoder(feature_column = 'WCIO Cause of Injury Code')),
    ('target_encoder_noi', MultipleTargetEncoder(feature_column = 'WCIO Nature of Injury Code')),
    ('target_encoder_pob', MultipleTargetEncoder(feature_column = 'WCIO Part Of Body Code')),
    
    # Calculate the days between the accident and assembly date new feature is called Days_between_Acc_Assembyl
    ('days_between_acc_ass', Days_between(start_col = 'Accident Date', end_col = 'Assembly Date')),
    
    # map column with mapping_dict to new structure, drop = true means that the new created column will replace the original
    ('mapper_carrier_type', ColumnMapper(column_name = 'Carrier Type', mapping_dict = carrier_type_mapping, drop_original = True)),

    # Use one Hot encoder/create dummies for the following 
    ('dummy_encoder', DummyEncoder(dummy_column = 'Attorney/Representative')),
    ('dummy_encoder_Carrier_Type', DummyEncoder(dummy_column = 'Carrier Type')),
    
    # encode na as 1 rest as 0
    ('na_indicator_C2', NAIndicatorEncoder('C-2 Date')),
    ('na_indicator_C3', NAIndicatorEncoder('C-3 Date')),
    ('na_indicator_C4', NAIndicatorEncoder('First Hearing Date'))
    ])

preprocess = preprocessing_pipeline.fit_transform(train, y = y_train)

In [15]:
preprocess

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Claim Identifier,...,WCIO Part Of Body Code_encoded_1. CANCELLED,WCIO Part Of Body Code_encoded_8. DEATH,WCIO Part Of Body Code_encoded_7. PTD,Days_between_Assembly Date_Accident Date,Attorney/Representative_Y,Carrier Type_Self-insured Private Entity,Carrier Type_Self-insured Public Entity,Carrier Type_Special Funds,Carrier Type_State Insurance Fund,Carrier Type_Unknown
0,2019-12-30,31.0,0,2020-01-01,0.00,1988.0,1,0,NEW HAMPSHIRE INSURANCE CO,5393875,...,0.006029,0.000861,0.000000,2,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-08-30,46.0,0,2020-01-01,1745.93,1973.0,1,1,ZURICH AMERICAN INSURANCE CO,5393091,...,0.009574,0.000000,0.000080,124,1.0,0.0,0.0,0.0,0.0,0.0
2,2019-12-06,40.0,0,2020-01-01,1434.80,1979.0,1,0,INDEMNITY INSURANCE CO OF,5393889,...,0.007057,0.001260,0.000756,26,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-12-30,61.0,0,2020-01-01,NaN,1958.0,1,0,STATE INSURANCE FUND,5393887,...,0.004018,0.000000,0.000028,2,0.0,0.0,0.0,0.0,1.0,0.0
4,2019-12-26,67.0,0,2020-01-01,0.00,1952.0,1,0,INDEMNITY INS. OF N AMERICA,5393863,...,0.009574,0.000000,0.000080,6,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570327,2022-12-14,35.0,0,2022-12-30,0.00,1987.0,1,0,STATE INSURANCE FUND,6165265,...,0.012409,0.000097,0.000232,16,0.0,0.0,0.0,0.0,1.0,0.0
570328,2022-12-15,33.0,0,2022-12-31,0.00,1989.0,1,0,WESCO INSURANCE CO,6165285,...,0.010212,0.000064,0.000032,16,0.0,0.0,0.0,0.0,0.0,0.0
570329,2022-12-13,61.0,0,2022-12-31,991.08,1961.0,1,0,SECURITY NATIONAL INSURANCE CO,6165506,...,0.010781,0.000000,0.000000,18,0.0,0.0,0.0,0.0,0.0,0.0
570330,2022-12-14,24.0,0,2022-12-31,0.00,1998.0,1,0,TECHNOLOGY INSURANCE CO. INC.,6165339,...,0.005979,0.000000,0.000120,17,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
train

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,...,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,...,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
4,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,...,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0
5,2019-12-26,67.0,N,2020-01-01,N,0.00,1952.0,2019-12-31,NaN,INDEMNITY INS. OF N AMERICA,...,31.0,"FALL, SLIP OR TRIP, NOC",10.0,CONTUSION,38.0,SHOULDER(S),11772,0.0,Not Work Related,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593451,2022-12-14,35.0,N,2022-12-30,N,0.00,1987.0,2022-12-30,NaN,STATE INSURANCE FUND,...,45.0,COLLISION OR SIDESWIPE WITH ANOTHER VEHICLE,10.0,CONTUSION,42.0,LOWER BACK AREA,10467,0.0,Not Work Related,4.0
593455,2022-12-15,33.0,N,2022-12-31,N,0.00,1989.0,2022-12-31,NaN,WESCO INSURANCE CO,...,74.0,"FELLOW WORKER, PATIENT OR OTHER PERSON",37.0,INFLAMMATION,35.0,HAND,11590,0.0,Not Work Related,6.0
593456,2022-12-13,61.0,N,2022-12-31,N,991.08,1961.0,2022-12-31,NaN,SECURITY NATIONAL INSURANCE CO,...,98.0,"CUMULATIVE, NOC",80.0,"ALL OTHER CUMULATIVE INJURY, NOC",34.0,WRIST,14227,0.0,Not Work Related,3.0
593457,2022-12-14,24.0,N,2022-12-31,N,0.00,1998.0,2022-12-31,NaN,TECHNOLOGY INSURANCE CO. INC.,...,59.0,USING TOOL OR MACHINERY,59.0,"ALL OTHER SPECIFIC INJURIES, NOC",55.0,ANKLE,10029,0.0,Not Work Related,5.0
